In [235]:
# Se importan librerias
import pandas as pd
import json
import numpy as np

In [236]:
# Ruta al archivo JSON
ruta_json = 'C:/Users/Mauro/Desktop/HENRY/Proyecto_individual/PI_1_MLOps/PI_MLOps-STEAM/steam_games.json/output_steam_games.json'

# Leer el archivo JSON y cargar los datos en una lista de diccionarios
data = []
with open(ruta_json, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            record = json.loads(line)
            data.append(record)
        except json.JSONDecodeError:
            # Manejar líneas no válidas según tu necesidad
            pass

# Crear un DataFrame a partir de la lista de diccionarios
df_steam_games = pd.DataFrame(data)

# Reemplazar NaN con None (opcional)
df_steam_games = df_steam_games.where(pd.notna(df_steam_games), None)

# Guardar el DataFrame como un archivo CSV
df_steam_games.to_csv('output_steam_games.csv', index=False)

In [237]:
# Se importa el dataset a analizar (previamente transformado)
#df_steam_games = pd.read_csv('output_steam_games.csv')

In [238]:
df_steam_games = df_steam_games.drop(['tags', 'specs', 'url', 'reviews_url'], axis=1)

In [239]:
# Dividir las cadenas de géneros en listas de géneros
#df_steam_games["genres"] = df_steam_games["genres"].str.split(",")

# Usar explode para descomponer la columna 'genres'
df_steam_games = df_steam_games.explode('genres')

# Eliminar filas con valores nulos en la columna 'genres'
df_steam_games = df_steam_games.dropna(subset=['genres'])

# Reemplazar los valores nulos (NaN) en la columna "genres" con listas vacías
#df_steam_games["genres"] = df_steam_games["genres"].apply(lambda x: [] if isinstance(x, float) else [genre.strip() for genre in x])


In [240]:
df_steam_games.head(5)

,publisher,genres,app_name,title,release_date,price,early_access,id,developer
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
88310,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
88310,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
88310,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
88310,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro


In [241]:
#Transformar las columnas "publisher", "app_name", "title", "url", "reviews_url" y "developer" en datos de tipo cadena (string)
columns_to_string = ["publisher", "app_name", "title", "developer"]

# Convierte las columnas especificadas a tipo string
df_steam_games[columns_to_string] = df_steam_games[columns_to_string].astype(str)


In [242]:
# Manejo de Datos Nulos: Eliminar filas que contienen todos los casilleros vacíos (NaN)
df_steam_games = df_steam_games.dropna(how='all')
df_steam_games.info()


<class 'pandas.core.frame.DataFrame'>
Index: 71554 entries, 88310 to 120443
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     71554 non-null  object
 1   genres        71554 non-null  object
 2   app_name      71554 non-null  object
 3   title         71554 non-null  object
 4   release_date  71501 non-null  object
 5   price         68355 non-null  object
 6   early_access  71554 non-null  object
 7   id            71552 non-null  object
 8   developer     71554 non-null  object
dtypes: object(9)
memory usage: 5.5+ MB


In [243]:
# Convierte la columna "release_date" a tipo datetime. Y convierte los valores no válidos en NaT (Not a Timestamp), que es una representación de valor nulo en datetime.
#df_steam_games["release_date"] = pd.to_datetime(df_steam_games["release_date"], errors="coerce")


In [244]:
# Convertir la columna 'release_date' al tipo datetime
#df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')

# Extraer el año y crear la columna 'release_year', asignando -1 a los valores no válidos
#df_steam_games['release_year'] = df_steam_games['release_date'].dt.year.fillna(-1).astype(int)

# Reemplazar los valores -1 por "Dato no disponible" en la columna 'release_year'
#df_steam_games['release_year'] = df_steam_games['release_year'].replace(-1, "Dato no disponible")

In [245]:
# Convertir la columna 'release_date' al tipo datetime y extraer el año
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')
df_steam_games['release_year'] = df_steam_games['release_date'].dt.year

# Eliminar las filas donde el 'release_year' es NaN (no válido)
df_steam_games.dropna(subset=['release_year'], inplace=True)

In [246]:
# Reemplazar "Dato no disponible" con NaN
df_steam_games['release_year'] = pd.to_numeric(df_steam_games['release_year'], errors='coerce')

# Convertir la columna 'release_year' a tipo entero
df_steam_games['release_year'] = df_steam_games['release_year'].astype('Int64')  # 'Int64' para aceptar NaN

In [247]:
df_steam_games = df_steam_games.drop(['release_date'], axis=1)

In [248]:
# Dividir las cadenas de géneros en listas de géneros
#df_steam_games["tags"] = df_steam_games["tags"].str.split(",")

# Reemplazar los valores nulos (NaN) en la columna "genres" con listas vacías
#df_steam_games["tags"] = df_steam_games["tags"].apply(lambda x: [] if isinstance(x, float) else [genre.strip() for genre in x])


In [249]:
#Transformar la columna "specs" en una lista de especificaciones técnicas del juego
# Define una función para manejar los elementos en la columna "specs"
#def process_specs(x):
#    if isinstance(x, list):
#        return [spec.strip() for spec in x]
#    elif pd.isna(x):
#        return []
#    else:
#        return [spec.strip() for spec in x.split(",")]

# Aplica la función para procesar la columna "specs"
#df_steam_games["specs"] = df_steam_games["specs"].apply(process_specs)

In [250]:
# Convierte la columna "price" a tipo float para representar el precio con decimales y los valores no válidos a NaN.
df_steam_games["price"] = pd.to_numeric(df_steam_games["price"], errors="coerce")


In [251]:
# Convierte la columna "early_access" a tipo bool
df_steam_games["early_access"] = df_steam_games["early_access"].astype(bool)


In [252]:
# Convierte la columna "id" a tipo int, tratando los valores no válidos como NaN
df_steam_games["id"] = pd.to_numeric(df_steam_games["id"], errors="coerce", downcast="integer")



In [253]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70704 entries, 88310 to 120443
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     70704 non-null  object 
 1   genres        70704 non-null  object 
 2   app_name      70704 non-null  object 
 3   title         70704 non-null  object 
 4   price         63523 non-null  float64
 5   early_access  70704 non-null  bool   
 6   id            70702 non-null  float64
 7   developer     70704 non-null  object 
 8   release_year  70704 non-null  Int64  
dtypes: Int64(1), bool(1), float64(2), object(5)
memory usage: 5.0+ MB


In [254]:
# Guardar el DataFrame transformado como un archivo CSV
df_steam_games.to_csv('output_steam_games_trans.csv', index=False)

In [255]:
df_steam_games.head(5)

,publisher,genres,app_name,title,price,early_access,id,developer,release_year
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018
88310,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018
88310,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018
88310,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018
88310,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018


In [256]:
df_steam_games.to_parquet('output_steam_games_trans.parquet', index=False)

In [257]:
# Dividir las listas de géneros en diferentes filas
#df_steam_games_exploded = df_steam_games.explode('genres')


In [258]:
df_steam_games.head(5)

,publisher,genres,app_name,title,price,early_access,id,developer,release_year
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018
88310,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018
88310,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018
88310,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018
88310,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140.0,Kotoshiro,2018


In [259]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70704 entries, 88310 to 120443
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     70704 non-null  object 
 1   genres        70704 non-null  object 
 2   app_name      70704 non-null  object 
 3   title         70704 non-null  object 
 4   price         63523 non-null  float64
 5   early_access  70704 non-null  bool   
 6   id            70702 non-null  float64
 7   developer     70704 non-null  object 
 8   release_year  70704 non-null  Int64  
dtypes: Int64(1), bool(1), float64(2), object(5)
memory usage: 5.0+ MB
